In [1]:
import time

import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm, tree
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, f1_score

In [2]:
coverages = ["A", "B", "AB"]
sections = [1, 2]
fields = ["10-G1", "13-D4"]

In [3]:
df = pd.read_csv("/content/drive/MyDrive/DOPO/Assegno di ricerca/6_ML/results.csv")

In [ ]:
def select_df(cov, coverage, sect, section, fiel, field):
  if coverage in cov and section == sect and field == fiel:
    return True
  else:
    return False

In [ ]:
def run_classifier(classifier, d, target):

  scoring = {'f1' : make_scorer(f1_score, average='macro')}
  scores = cross_validate(classifier, X, y, scoring=scoring)

  return scores["test_f1"]

In [ ]:
def run_ML(clsf, data, type_val):

  r = dict()

  features= ['cand', 'co-au',
             'cand_comm', 'comm_cand',
             'BC', 'CC',
             'cand_other', 'other_cand',
             'I1', 'I2', 'I3']
  
  r["no type"] = run_classifier(clsf, data[features], d['esito'])

  features.append('books')
  r["+books"] = run_classifier(clsf, data[features], d['esito'])
  features.pop()

  features.append('articles')
  r["+articles"] = run_classifier(clsf, data[features], d['esito'])
  features.pop()

  features.append('other_pubbs')
  r["+other_pubbs"] = run_classifier(clsf, data[features], d['esito'])

  features.append('books')
  r["+other_pubbs+books"] = run_classifier(clsf, data[features], d['esito'])
  features.pop()

  features.append('articles')
  r["+other_pubbs+articles"] = run_classifier(clsf, data[features], d['esito'])
  features.pop(-2)

  features.append('books')
  r["+articles+books"]  = run_classifier(clsf, data[features], d['esito'])

  features.append('other_pubbs')
  r["with type"] = run_classifier(clsf, data[features], d['esito'])

  return r

In [ ]:
def results_dict(clf_type, dataframe, cov_list, sect_list, field_list)

  ML = dict()

  #inizializzazione classifier
  if clf_type == "SVM":
    clf = svm.SVC(kernel='linear', C=1, random_state=0)
  elif clf_type == "Tree"
    clf = tree.DecisionTreeClassifier()
  
  #selezione porzione di dati
    for cov in cov_list:
      ML[cov] = dict()
      for sect in sect_list:
        ML[cov][sect] = dict()
        for fiel in field_list:
          ML[cov][sect][fiel] = dict()
          data = dataframe.loc[dataframe.apply(lambda x: select_df(cov, x["coverage"], sect, x["section"], field, x["field"]), axis=1)]

          #porzione solo ANVUR
          ML[cov][sect][fiel]["ANVUR"] = run_classifier(clf, data[['I1', 'I2', 'I3']], data['esito'])

          #porzione nostri indicatori
          ML[cov][sect][fiel].extend(run_ML(clf, data))

In [ ]:
SVM_results = results_dict("SVM", df, coverages, sections, fields)

In [ ]:
Tree_results = results_dict("Tree", df, coverages, sections, fields)